In [1]:
import numpy as np
from scipy import io as sio
from matplotlib import pyplot as plt
import inspect

In [2]:
import typhon as tp

/home/user/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
import typhon.retrieval as tpr

In [5]:
import typhon.retrieval.bmci as tprb

In [6]:
import typhon.retrieval.mcmc as tprm

/home/user/anaconda3/lib/python3.6/site-packages/typhon/arts/workspace/__init__.py:101: UserWarning: ARTS_BUILD_PATH environment variable required to locate ARTS API.
  warn("ARTS_BUILD_PATH environment variable required to locate ARTS API.")


ImportError: cannot import name 'Workspace'

In [8]:
import typhon.retrieval.qrnn as tprq

In [9]:
# Define a function to easily see the source code of a function. 
def showfunc(functiontoshow):
    print(inspect.getsource(functiontoshow))

In [11]:
showfunc(tprb.BMCI.cdf)

    def cdf(self, y_obs, x2_max = -1):
        r"""
        A posteriori cumulative distribution function (CDF).

        This function approximates the cumulative posterior distribution
        :math:`F(x | \mathbf{y})` for the given observation `y_obs` using

        .. math::

            F(x | \mathbf{y}) = \int_{-\infty}^{x} p(x' | \mathbf{y}) \: dx'
            \approx \sum_{x_i < x} \frac{w_i(\mathbf{y})}{\sum_j w_j(\mathbf{y})}

        Args:

            y_obs(numpy.array): `m`-element array containing the observation
                                 for which to compute the posterior CDF.

            x2_max(float): The :math:`\chi^2` cutoff to apply to elements in the
                           database. Ignored if less than zero.

        Returns:

            A tuple `(xs, ys)` containing the estimated values of the posterior
            CDF :math:`F(x | \mathbf{y})` evaluated at the $x$ values
            corresponding to the hits in the database.

        Raises:

      

In [12]:
J = tp.arts.xml.load("./ClearSky_1D_jacobian.xml")
J.shape

(22, 50)

In [ ]:
Tb_simulated = tp.arts.xml.load("./ClearSky_1D_Tb.xml")
Tb_BosungObs = np.array([26.193, 23.754, 22.435, 18.510, 15.617, 12.566, 12.388, 11.472, 
                         107.09, 123.677, 147.536, 178.933, 216.787, 250.225, 273.193, 282.353, 
                         286.576, 288.430, 289.69, 290.449, 291.361, 291.436])